> ### TP numéro 4 Base de Données - Alex FULBERT

In [2]:
import sqlite3 as sql
import pandas as pd 

#### QUESTION 1
Connectez-vous à la base « TP_python_sqlite.db » fournie sur le site pédagogique, en la recopiant au préalable dans votre répertoire.

In [3]:
# Question 2 
conn = sql.connect('TP_python_sqlite.db')
curs = conn.cursor()

#### QUESTION 2

Faire afficher par la requête «select * from MAGASIN» le contenu de la table «MAGASIN» qui est déjà créée dans la base (fournir une solution avec la fonction fetchall puis une autre avec la fonction fetchone)

In [17]:
data = curs.execute('SELECT * FROM MAGASIN;').fetchall()
data

[(1, 'New York', 'New York', 'NY'),
 (2, 'Chicago', 'Cook', 'IL'),
 (3, 'Atlanta', 'Fulton', 'GA'),
 (4, 'Los Angeles', 'Los Angeles', 'CA'),
 (5, 'San Francisco', 'San Francisco', 'CA'),
 (6, 'Philadelphia', 'Philadelphia', 'PA'),
 (7, 'Pittsburgh', 'Allegheny', 'PA'),
 (8, 'New Orleans', 'Orleans', 'LA'),
 (9, 'Seattle', 'King', 'WA'),
 (10, 'Dallas', 'Dallas', 'TX'),
 (11, 'Cincinnati', 'Hamilton', 'OH'),
 (12, 'Minneapolis', 'Hennepin', 'MN'),
 (14, 'Phoenix', 'Maricopa', 'AZ'),
 (15, 'Denver', 'Denver', 'CO'),
 (16, 'St. Louis', 'St. Louis', 'MO'),
 (17, 'Washington', 'DC', 'DC'),
 (18, 'Miami', 'Dade', 'FL'),
 (19, 'Boston', 'Suffolk', 'MA'),
 (20, 'Nashville', 'Davidson', 'TN')]

#### QUESTION 3

Créer un dataframe à partir du résultat de la requête « select * from MAGASIN » et faire afficher le contenu du dataframe.

In [11]:
data = curs.execute('SELECT * FROM MAGASIN;').fetchall()
cols = curs.execute('SELECT * FROM MAGASIN;')
name_columns = [description[0] for description in cols.description]
df_mag = pd.DataFrame(data, columns= name_columns)
df_mag = df_mag[['NOM', 'VILLE', 'COMTE', 'ETAT']]
df_mag.head(3)

NOM     VILLE     COMTE ETAT
0  Store No. 1  New York  New York   NY
1  Store No. 2   Chicago      Cook   IL
2  Store No. 3   Atlanta    Fulton   GA

#### QUESTION 4

Créer une table «employe» avec 2 colonnes : «nom» de type TEXT et «age» de type INTEGER, de clé primaire « nom ». Bien créer la table depuis Python dans le Notebook et non pas avec sqlite3.

In [4]:
employe = curs.execute('CREATE TABLE employe(nom varchar(15) PRIMARY KEY, age integer);')

#### QUESTION 5

Insérer dans la table « employe » le n-uplet avec les valeurs « toto » et « 25 ».

In [4]:
insert = curs.execute('INSERT INTO employe VALUES ("toto", 25);') # La mise à jour n'est pas visible sur DB Browser

#### QUESTION 6

Faire afficher le contenu de la table « employe » (requête « select * from employe ; »). Ouvrir une fenêtre sqlite3 ou DB Browser for SQLite : la mise à
jour est-elle visible ?

In [18]:
curs.execute('SELECT * FROM employe;').fetchall() # Pas visible sur DB Browser !!!!

[('toto', 25), ('alex', 27)]

#### QUESTION 7

Rendre permanente la mise à jour effectuée.

In [7]:
# Rendons l'insert permanent 
curs.execute('DELETE FROM employe WHERE nom = "toto"') 
req1 = 'INSERT INTO employe VALUES ("toto", 25);'
insert2 = curs.execute(req1)
conn.commit()

#### QUESTION 8 / 9

Ecrire un programme qui demande à saisir un nom et un âge, et qui insère les informations
saisies dans la table « employe » ; rendre permanente la mise à jour effectuée.

Ecrire un programme qui demande à saisir un état et qui exécute la requête retrouvant tous
les magasins situés dans cet état (requête SELECT paramétrée sur la table MAGASIN).

In [8]:
def add_employe(nom, age):
    
    req1 = 'INSERT INTO employe VALUES (?, ?);'
    insert2 = curs.execute(req1,(nom,age))
    conn.commit()
    
def magasin_etat(etat):
    req1 = 'select * from MAGASIN where ETAT = ?'
    magasins_etat = curs.execute(req1,(etat,))
    return magasins_etat.fetchall()


add_employe('alex', 27)
magasin_etat('NY')

[('Store No. 1',
  '999 Main Street',
  'New York',
  'New York',
  'NY',
  'Jones',
  'Modern')]

## Création d’un schéma en étoile pour stocker un cube de données

In [12]:
# CONSTRUCTION D'UNE NOUVELLE TABLE MAGASIN 

df_mag = df_mag[['NOM', 'VILLE', 'COMTE', 'ETAT']]
df_mag['NOM'] = df_mag['NOM'].apply(lambda x : x.split('.')[-1].strip())
df_mag.rename(columns={'NOM' : 'ID_MAGASIN'}, inplace= True)
df_mag['ID_MAGASIN'] = df_mag['ID_MAGASIN'].astype(int)

array = df_mag.to_numpy()
tuples = [tuple(elem) for elem in array]

req1 = 'select * from MAGASIN' 
req2 = 'DROP TABLE MAGASIN '
req3 = 'CREATE TABLE MAGASIN(ID_MAGASIN INTEGER PRIMARY KEY, VILLE varchar(15), COMTE varchar(15), ETAT varchar(15))'
req4 = 'INSERT INTO MAGASIN VALUES(?,?,?,?);'

curs.execute(req2)
curs.execute(req3)
curs.executemany(req4, tuples)

data = curs.execute('SELECT * FROM MAGASIN;').fetchall()
cols = curs.execute('SELECT * FROM MAGASIN;')
name_columns = [description[0] for description in cols.description]
df_mag = pd.DataFrame(data, columns= name_columns)
df_mag.head(3)
#curs.execute(req1).fetchall()

ID_MAGASIN     VILLE     COMTE ETAT
0           1  New York  New York   NY
1           2   Chicago      Cook   IL
2           3   Atlanta    Fulton   GA

In [13]:
# CONSTRUCTION D'UNE NOUVELLE TABLE PRODUIT 

data = curs.execute('SELECT * FROM PRODUIT;').fetchall()
cols = curs.execute('SELECT * FROM PRODUIT;')
name_columns = [description[0] for description in cols.description]

df_produit = pd.DataFrame(data, columns = name_columns)
df_produit = df_produit[['CODE_PRODUIT', 'LIBELLE', 'SOUS_CATEGORIE', 'CATEGORIE', 'MARQUE']]
df_produit['CODE_PRODUIT'] = df_produit['CODE_PRODUIT'].astype(int)
df_produit.rename(columns={'CODE_PRODUIT' : 'ID_PRODUIT'}, inplace= True)

array = df_produit.to_numpy()
tuples = [tuple(elem) for elem in array]

req1 = 'select * from PRODUIT' 
req2 = 'DROP TABLE PRODUIT '
req3 = 'CREATE TABLE PRODUIT(ID_PRODUIT integer primary key, LIBELLE varchar(15), SOUS_CATEGORIE varchar(15), CATEGORIE varchar(15), MARQUE varchar(15))'
req4 = 'INSERT INTO PRODUIT VALUES(?,?,?,?,?);'

curs.execute(req2)
curs.execute(req3)
curs.executemany(req4, tuples)
#curs.execute(req1).fetchall()

data = curs.execute('SELECT * FROM PRODUIT;').fetchall()
cols = curs.execute('SELECT * FROM PRODUIT;')
name_columns = [description[0] for description in cols.description]
df_prod = pd.DataFrame(data, columns= name_columns)
df_prod.head(3)

ID_PRODUIT             LIBELLE SOUS_CATEGORIE CATEGORIE        MARQUE
0           1        Lasagna 6 oz   Frozen Foods      Food  Cold Gourmet
1           2      Beef Stew 6 oz   Frozen Foods      Food  Cold Gourmet
2           3  Turkey Dinner 8 oz   Frozen Foods      Food   Frozen Bird

In [14]:
# CONSTRUCTION D'UNE NOUVELLE TABLE PROMOTION 

data = curs.execute('SELECT * FROM PROMOTION;').fetchall()
cols = curs.execute('SELECT * FROM PROMOTION;')
name_columns = [description[0] for description in cols.description]

df_promotion = pd.DataFrame(data, columns = name_columns)
df_promotion = df_promotion[['NOM_PROMOTION', 'TYPE_PUBLICITE']]

array = df_promotion.to_numpy()
tuples = [tuple(elem) for elem in array]

req1 = 'SELECT * FROM PROMOTION' 
req2 = 'DROP TABLE PROMOTION '
req3 = 'CREATE TABLE PROMOTION(ID_PROMOTION integer primary key autoincrement, NOM_PROMOTION varchar(15), TYPE_PUBLICITE varchar(15))'
req4 = 'INSERT INTO PROMOTION(NOM_PROMOTION, TYPE_PUBLICITE) VALUES(?,?);'

curs.execute(req2)
curs.execute(req3)
curs.executemany(req4, tuples)
#curs.execute(req1).fetchall()

data = curs.execute('SELECT * FROM PROMOTION;').fetchall()
cols = curs.execute('SELECT * FROM PROMOTION;')
name_columns = [description[0] for description in cols.description]
df_promo = pd.DataFrame(data, columns= name_columns)
df_promo.head(3)

ID_PROMOTION          NOM_PROMOTION   TYPE_PUBLICITE
0             1  Blue Ribbon Discounts      Daily Paper
1             2    Red Carpet Closeout     Sunday Paper
2             3               Ad Blitz  Paper and Radio

In [15]:
# CONSTRUCTION DE LA TABLE TEMPS À PARTIR DE LA TABLE VENTE

data = curs.execute('SELECT * FROM VENTES;').fetchall()
cols = curs.execute('SELECT * FROM VENTES;')
name_columns = [description[0] for description in cols.description]
df_ventes = pd.DataFrame(data, columns = name_columns)
df_ventes.rename(columns={'JOUR' : 'J'}, inplace= True)
df_ventes['J'] = pd.to_datetime(df_ventes['J'])
df_ventes['JOUR'] = df_ventes['J'].dt.day
df_ventes['MOIS'] = df_ventes['J'].dt.month
df_ventes['ANNEE'] = df_ventes['J'].dt.year
df_ventes[['ANNEE','JOUR', 'MOIS']] = df_ventes[['ANNEE','JOUR', 'MOIS']].astype(str)

array = df_ventes[['JOUR', 'MOIS', 'ANNEE']].to_numpy()
tuples = [tuple(elem) for elem in array]
#req2 = 'DROP TABLE TEMPS '
req3 = 'CREATE TABLE TEMPS(ID_TEMPS integer primary key autoincrement, JOUR INTEGER, MOIS INTEGER, ANNEE INTEGER)'
req4 = 'INSERT INTO TEMPS(JOUR, MOIS, ANNEE) VALUES(?,?,?);'
#curs.execute(req2)
curs.execute(req3)
curs.executemany(req4, tuples)

data = curs.execute('SELECT * FROM TEMPS;').fetchall()
cols = curs.execute('SELECT * FROM TEMPS;')
name_columns = [description[0] for description in cols.description]
df_temps = pd.DataFrame(data, columns= name_columns)
df_temps.head(3)

ID_TEMPS  JOUR  MOIS  ANNEE
0         1     3    10   1994
1         2     3    10   1994
2         3     3    10   1994

In [16]:
# LA TABLE DE FAITS 

req1 = 'CREATE TABLE FAITS( \
	id_magasin INTEGER REFERENCES MAGASIN(ID_NOM),\
	id_produit INTEGER REFERENCES PRODUIT(ID_PRODUIT),\
	id_promotion INTEGER REFERENCES PROMOTION(ID_PROMOTION),\
	id_temps INTEGER REFERENCES TEMPS(ID_TEMPS),\
	Quantite INTEGER,\
	CA INTEGER\
);'
curs.execute(req1)
conn.commit() # RENDRE TOUTES LES MODIFICATIONS PERMANENTES, VISIBLE SUR DB BROWSER

In [ ]:
# CELLULE NON UTILISE : JE VOULAIS M'EN SERVIR POUR REMPLIR LA TABLE DES FAITS 

df_ventes['NOM_MAGASIN'] = df_ventes['NOM_MAGASIN'].apply(lambda x : x.split('.')[-1].strip())
df_ventes.rename(columns={'NOM_MAGASIN' : 'ID_NOM', 'CODE_PRODUIT' : 'id_produit', 'PROMOTION' : 'nom_promotion' }, inplace = True)
df_ventes[['ANNEE','JOUR', 'MOIS']] = df_ventes[['ANNEE','JOUR', 'MOIS']].astype(str)
df_ventes.drop(columns= ['J', 'JOUR', 'MOIS', 'ANNEE'], inplace = True)
df_ventes = df_ventes.merge(df_promotion[['id_promotion', 'nom_promotion']], on = ['nom_promotion'])
#df_ventes[['ANNEE','JOUR', 'MOIS']] = df_ventes[['ANNEE','JOUR', 'MOIS']].astype(int)
#df_ventes = df_ventes.merge(df_temps, on = ['JOUR','MOIS','ANNEE'])
df_ventes.drop(columns= ['nom_promotion'], inplace = True)
df_ventes['id_temps'] = df_ventes.index + 1 

df_ventes